### Imports

In [3]:
## Libraries
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import statistics
import torchvision.models as models
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

## Python Files
from utils
from datas

### Convert silly .mat type to .png images

In [4]:
utils.convertMat2Png()

ERROR:root:ERROR: MATLAB type not supported: containers.Map, (uint32)


### Load datas